In [55]:
import os
import sys

# 自动检测环境
IS_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ
competition_name = 'titanic/'

if IS_KAGGLE:
    print("Running on Kaggle")
    DATA_PATH = '/kaggle/input/' + competition_name
    OUTPUT_PATH = '/kaggle/working'
    # 可能需要安装包
else:
    print("Running locally")
    DATA_PATH = './data'
    OUTPUT_PATH = './output'

Running locally


In [56]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [57]:
# Add src directory to Python path and import model
if IS_KAGGLE:
    !git clone https://github.com/catathome0410/Titanic_practice.git
    import sys
    sys.path.append('/kaggle/working/Titanic_practice/src')
    
    from model import simple_logr_scaler, log_R_solver, NN_solver
else:
    sys.path.append('./src')
    from model import simple_logr_scaler, log_R_solver, NN_solver

In [58]:
train = pd.read_csv(DATA_PATH + '/train.csv')
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [59]:
df_train = train.copy()
df_train['Embarked'] = df_train['Embarked'].fillna('S')
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())
df_train = df_train.drop(['Name', 'Ticket'], axis = 1)
df_train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,7.2500,NaN,S
1,2,1,1,female,38.0,1,0,71.2833,C85,C
2,3,1,3,female,26.0,0,0,7.9250,NaN,S
3,4,1,1,female,35.0,1,0,53.1000,C123,S
4,5,0,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,NaN,S
887,888,1,1,female,19.0,0,0,30.0000,B42,S
888,889,0,3,female,28.0,1,2,23.4500,NaN,S
889,890,1,1,male,26.0,0,0,30.0000,C148,C


In [60]:
df_train['cabin_info_null'] = df_train['Cabin'].isna()
df_train['cabin_info_null'] = df_train['cabin_info_null'].astype('int32')
df_train = df_train.drop(['Cabin'], axis = 1)
df_train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,cabin_info_null
0,1,0,3,male,22.0,1,0,7.2500,S,1
1,2,1,1,female,38.0,1,0,71.2833,C,0
2,3,1,3,female,26.0,0,0,7.9250,S,1
3,4,1,1,female,35.0,1,0,53.1000,S,0
4,5,0,3,male,35.0,0,0,8.0500,S,1
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,S,1
887,888,1,1,female,19.0,0,0,30.0000,S,0
888,889,0,3,female,28.0,1,2,23.4500,S,1
889,890,1,1,male,26.0,0,0,30.0000,C,0


In [61]:
df_train_d = pd.get_dummies(df_train, columns=['Sex', 'Embarked'], dtype='int32')
df_train_d = df_train_d.drop('Sex_female', axis=1)
df_train_d

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,cabin_info_null,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,1,1,0,0,1
1,2,1,1,38.0,1,0,71.2833,0,0,1,0,0
2,3,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,4,1,1,35.0,1,0,53.1000,0,0,0,0,1
4,5,0,3,35.0,0,0,8.0500,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,27.0,0,0,13.0000,1,1,0,0,1
887,888,1,1,19.0,0,0,30.0000,0,0,0,0,1
888,889,0,3,28.0,1,2,23.4500,1,0,0,0,1
889,890,1,1,26.0,0,0,30.0000,0,1,1,0,0


In [62]:
X_train = np.array(df_train_d.drop(['Survived', 'PassengerId'], axis = 1))
Y_train = np.array(df_train_d['Survived'])

In [ ]:
n_pick_0 = np.floor(X_train.shape[0] / 16)
print(n_pick_0)
n_pick = 54 * 16

55.0


In [ ]:
slr = simple_logr_scaler(X_train)
X_transform = slr.fit()
Y_transform = Y_train

Xs = X_transform[:n_pick, :]
Ys = Y_transform[:n_pick]

In [68]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3, shuffle=True, random_state=42)

In [69]:
model_accuracies = []

for train_idx, test_idx in kf.split(Xs):
    Xss_train, Xss_test = Xs[train_idx], Xs[test_idx]
    yss_train, yss_test = Ys[train_idx], Ys[test_idx]
    
    model = log_R_solver(Xss_train, yss_train, alpha = 0.02, lambda2=0.1)
    model.fit_L2()
    
    y_pred = model.transfrom(Xss_test)
    accuracy = np.sum(yss_test == y_pred) / yss_test.shape[0]
    model_accuracies.append(accuracy)

print(f"平均 MSE: {np.mean(model_accuracies):.4f}")
print(f"MSE 标准差: {np.std(model_accuracies):.4f}")

iteration No.1000, likely_hood = -11.877926073280008, likely_hood_ratio = 0.009141708911967568, norm(theta) = 2.17271465, norm(grad) = 0.00109696, grad/theta ratio = 0.000505
iteration No.2000, likely_hood = -10.888254943989603, likely_hood_ratio = 0.013672027714816272, norm(theta) = 2.97795328, norm(grad) = 0.00072637, grad/theta ratio = 0.000244
iteration No.3000, likely_hood = -10.425517699923933, likely_hood_ratio = 0.01712384417464683, norm(theta) = 3.57104059, norm(grad) = 0.00059954, grad/theta ratio = 0.000168
iteration No.4000, likely_hood = -10.131981757726376, likely_hood_ratio = 0.02015125611868661, norm(theta) = 4.08393427, norm(grad) = 0.00054197, grad/theta ratio = 0.000133
iteration No.5000, likely_hood = -9.919510888497223, likely_hood_ratio = 0.02299724884646721, norm(theta) = 4.56289949, norm(grad) = 0.00051427, grad/theta ratio = 0.000113
iteration No.6000, likely_hood = -9.754549033071672, likely_hood_ratio = 0.025778536655065666, norm(theta) = 5.02962511, norm(gra

In [70]:
model_accuracies

[0.75, 0.75, 0.6875]

In [71]:
df_test = pd.read_csv(DATA_PATH + '/test.csv')
df_test['Embarked'] = df_train['Embarked'].fillna('S')
df_test['Age'] = df_test['Age'].fillna(df_train['Age'].median())
df_test = df_test.drop(['Name', 'Ticket'], axis = 1)
df_test['cabin_info_null'] = df_test['Cabin'].isna()
df_test['cabin_info_null'] = df_test['cabin_info_null'].astype('int32')
df_test = df_test.drop(['Cabin'], axis = 1)
df_test_d = pd.get_dummies(df_test, columns=['Sex', 'Embarked'], dtype='int32')
df_test_d = df_test_d.drop('Sex_female', axis=1)
X_test = np.array(df_test_d.drop(['PassengerId'], axis = 1))
Xtsts = slr.transform(X_test)

## if necessary, train a logR model on all data 
# Yres = lrsr.transfrom(Xtsts)
# df_test['Survived'] = Yres
# df_test

In [72]:
if IS_KAGGLE == False:
    os.makedirs('output', exist_ok=True)


In [ ]:
# df_res = df_test[['PassengerId', 'Survived']]
# df_res.to_csv(OUTPUT_PATH + '/v4_l2_reg.csv', index=False, encoding='utf-8')

In [ ]:
kf = KFold(n_splits=9, shuffle=True, random_state=42)
model_accuracies = []

for train_idx, test_idx in kf.split(Xs):
    Xss_train, Xss_test = Xs[train_idx], Xs[test_idx]
    yss_train, yss_test = Ys[train_idx], Ys[test_idx]
    
    model = NN_solver(Xss_train, yss_train, alpha=0.02, n_seed = 200, output_gap = 100)
    model.fit_shallow_parallel(shallow_iter_limit = 2000, target_loss = 320)

    n_min_seed = model.loss_arr.argmin()
    model.fit_deep(10000, lr_raito = 0.1, use_input = True, W1_i=model.W1[:, :, n_min_seed], b1_i=model.b1[:, :, n_min_seed], W2_i=model.W2[:, :, n_min_seed], b2_i=model.b2[:, :, n_min_seed], W3_i=model.W3[:, :, n_min_seed], b3_i=model.b3[:, :, n_min_seed])
    
    accuracy = model.check_performance(Xss_test, yss_test)
    model_accuracies.append(accuracy)

print(f"平均 accuracy: {np.mean(model_accuracies):.4f}")
print(f"accuracy 标准差: {np.std(model_accuracies):.4f}")

epoch No.1, min loss = 24.317504, from seed No.88
epoch No.101, min loss = 16.319368, from seed No.143
epoch No.201, min loss = 12.850969, from seed No.167
epoch No.301, min loss = 11.355266, from seed No.167
epoch No.401, min loss = 10.545654, from seed No.167
epoch No.501, min loss = 9.976629, from seed No.167
epoch No.601, min loss = 9.605617, from seed No.167
epoch No.701, min loss = 9.325693, from seed No.167
epoch No.801, min loss = 9.120476, from seed No.167
epoch No.901, min loss = 8.960258, from seed No.167
epoch No.1001, min loss = 8.811155, from seed No.71
epoch No.1101, min loss = 8.623361, from seed No.71
epoch No.1201, min loss = 8.472784, from seed No.71
epoch No.1301, min loss = 8.367549, from seed No.71
epoch No.1401, min loss = 8.277628, from seed No.71
epoch No.1501, min loss = 8.221985, from seed No.71
epoch No.1601, min loss = 8.182179, from seed No.71
epoch No.1701, min loss = 8.158791, from seed No.71
epoch No.1801, min loss = 8.148655, from seed No.71
epoch No.1

/Users/nanyidong/development/Kaggle/Titanic/./src/model.py:572: RuntimeWarning: divide by zero encountered in log
  self.loss_arr = - np.sum(self.Y[np.newaxis, :, np.newaxis] * np.log(a3_e) + (1-self.Y[np.newaxis, :, np.newaxis]) * np.log(1-a3_e), axis=1)
/Users/nanyidong/development/Kaggle/Titanic/./src/model.py:572: RuntimeWarning: invalid value encountered in multiply
  self.loss_arr = - np.sum(self.Y[np.newaxis, :, np.newaxis] * np.log(a3_e) + (1-self.Y[np.newaxis, :, np.newaxis]) * np.log(1-a3_e), axis=1)
/Users/nanyidong/development/Kaggle/Titanic/./src/model.py:797: RuntimeWarning: divide by zero encountered in log
  loss = - np.sum(self.Y.T * np.log(a3_e) + (1-self.Y.T) * np.log(1-a3_e))
/Users/nanyidong/development/Kaggle/Titanic/./src/model.py:797: RuntimeWarning: invalid value encountered in multiply
  loss = - np.sum(self.Y.T * np.log(a3_e) + (1-self.Y.T) * np.log(1-a3_e))


epoch No.3901, min loss = nan, from seed No.26
epoch No.4001, min loss = nan, from seed No.26
epoch No.1, loss = nan
epoch No.101, loss = nan
epoch No.201, loss = nan
epoch No.301, loss = nan
epoch No.401, loss = nan
epoch No.501, loss = nan
epoch No.601, loss = nan
epoch No.701, loss = nan
epoch No.801, loss = nan
epoch No.901, loss = nan
epoch No.1001, loss = nan
epoch No.1101, loss = nan
epoch No.1201, loss = nan
epoch No.1301, loss = nan
epoch No.1401, loss = nan
epoch No.1501, loss = nan
epoch No.1601, loss = nan
epoch No.1701, loss = nan
epoch No.1801, loss = nan
epoch No.1901, loss = nan
epoch No.2001, loss = nan
epoch No.2101, loss = nan
epoch No.2201, loss = nan
epoch No.2301, loss = nan
epoch No.2401, loss = nan
epoch No.2501, loss = nan
epoch No.2601, loss = nan
epoch No.2701, loss = nan
epoch No.2801, loss = nan
epoch No.2901, loss = nan
epoch No.3001, loss = nan
epoch No.3101, loss = nan
epoch No.3201, loss = nan
epoch No.3301, loss = nan
epoch No.3401, loss = nan
epoch No

In [74]:
nslr = NN_solver(Xs, Ys, alpha=0.02, n_seed = 200, output_gap = 100)
nslr.fit_shallow_parallel(shallow_iter_limit = 2000, target_loss = 320)
n_min_seed = nslr.loss_arr.argmin()
nslr.fit_deep(10000, lr_raito = 0.1, use_input = True, W1_i=model.W1[:, :, n_min_seed], b1_i=model.b1[:, :, n_min_seed], W2_i=model.W2[:, :, n_min_seed], b2_i=model.b2[:, :, n_min_seed], W3_i=model.W3[:, :, n_min_seed], b3_i=model.b3[:, :, n_min_seed])
    

epoch No.1, min loss = 27.284826, from seed No.173
epoch No.101, min loss = 16.934086, from seed No.125
epoch No.201, min loss = 13.864741, from seed No.8
epoch No.301, min loss = 12.191434, from seed No.8
epoch No.401, min loss = 10.667739, from seed No.40
epoch No.501, min loss = 9.781636, from seed No.40
epoch No.601, min loss = 9.198866, from seed No.10
epoch No.701, min loss = 8.528440, from seed No.10
epoch No.801, min loss = 7.868302, from seed No.10
epoch No.901, min loss = 7.352263, from seed No.10
epoch No.1001, min loss = 6.968447, from seed No.10
epoch No.1101, min loss = 6.661791, from seed No.10
epoch No.1201, min loss = 6.408439, from seed No.10
epoch No.1301, min loss = 6.213098, from seed No.10
epoch No.1401, min loss = 6.067694, from seed No.10
epoch No.1501, min loss = 5.958401, from seed No.10
epoch No.1601, min loss = 5.885227, from seed No.10
epoch No.1701, min loss = 5.839555, from seed No.10
epoch No.1801, min loss = 5.813819, from seed No.10
epoch No.1901, min 

/Users/nanyidong/development/Kaggle/Titanic/./src/model.py:572: RuntimeWarning: divide by zero encountered in log
  self.loss_arr = - np.sum(self.Y[np.newaxis, :, np.newaxis] * np.log(a3_e) + (1-self.Y[np.newaxis, :, np.newaxis]) * np.log(1-a3_e), axis=1)
/Users/nanyidong/development/Kaggle/Titanic/./src/model.py:572: RuntimeWarning: invalid value encountered in multiply
  self.loss_arr = - np.sum(self.Y[np.newaxis, :, np.newaxis] * np.log(a3_e) + (1-self.Y[np.newaxis, :, np.newaxis]) * np.log(1-a3_e), axis=1)


epoch No.3701, min loss = nan, from seed No.89
epoch No.3801, min loss = nan, from seed No.89
epoch No.3901, min loss = nan, from seed No.89
epoch No.4001, min loss = nan, from seed No.89
epoch No.1, loss = 13.027176
epoch No.101, loss = 11.954694
epoch No.201, loss = 11.893742
epoch No.301, loss = 11.875829
epoch No.401, loss = 11.877319
epoch No.501, loss = 11.851087
epoch No.601, loss = 11.825099
epoch No.701, loss = 11.815187
epoch No.801, loss = 11.786405
epoch No.901, loss = 11.763045
epoch No.1001, loss = 11.744957
epoch No.1101, loss = 11.718282
converged


True

In [75]:
Y_res_nn = nslr.predict(Xtsts)
df_test['Survived'] = Y_res_nn
df_res = df_test[['PassengerId', 'Survived']]
df_res.to_csv(OUTPUT_PATH + 'v10_nn.csv', index=False, encoding='utf-8')

In [ ]:
    # 就 NN的效果来说已经很好了，接下来要提升titanic 的数据效果是ML hyper parametr tuning 和feature engineering 方面的工作
    
    ## NN variance tuning, 剪枝，regularization, early stopping 这些
## K-fold training, 
## Roc curve, precision / recall 
    ## 把Titanic 剩下的几个feature 挖出点东西来. Name / Cabin / 
    ## Bias / Variance curve